In [ ]:
import pandas as pd
import os
from datetime import datetime
import mydashtools as dt
import plotly.express as px
pd.options.plotting.backend = "plotly"
if dt.is_notebook():
    __file__ = "notebook/"

events = pd.read_csv(os.path.dirname(os.path.realpath(__file__))+"/../events/barco_events_all.csv", sep=";",decimal=',',low_memory=False, index_col=False)
events['date'] = pd.to_datetime(events['date'])
events.set_index('date', inplace=True)

if dt.is_notebook():
    print(events.head())

In [ ]:
limit_date = datetime.now().replace(month=datetime.now().month - 6)
events_par_semaine = events[events.index > str(limit_date)[0:19]]
events_par_semaine = events_par_semaine.resample('W')[['url']].count().rename(columns={'url': 'Nb évènements'})

events_par_semaine["Nb d'évènements"] = events_par_semaine[events_par_semaine.index <= datetime.now().isoformat()]["Nb évènements"]
events_par_semaine["Nb d'évènements futurs"] = events_par_semaine[events_par_semaine.index > datetime.now().isoformat()]["Nb évènements"]
del events_par_semaine["Nb évènements"]
fig = px.bar(events_par_semaine,color_discrete_map={"Nb d'évènements": '#636efa', "Nb d'évènements futurs": '#9aa1fa'})
fig.update_layout(showlegend=False,xaxis_title=None,yaxis_title=None)
fig.update_yaxes(dtick="n")

dt.fig_save_or_show(fig, 'evenements_01_events_par_semaine')

In [ ]:
events_mensuels_par_an = events.resample('MS')[['url']].count().rename(columns={'url': 'Nb d\'évènements'})

fig = px.bar(events_mensuels_par_an)
fig.update_layout(showlegend=False,xaxis_title=None,yaxis_title=None)
dt.fig_save_or_show(fig, 'evenements_02_events_mensuels_par_an')

In [ ]:
events_par_an = events.resample('YS')[['url']].count().rename(columns={'url': 'Nb d\'évènements'})
events_par_an.reset_index(inplace=True)
events_par_an['Année'] = events_par_an['date'].apply(lambda x: str(x)[0:4])
events_par_an.set_index('Année', inplace=True)
del events_par_an['date']

fig = px.bar(events_par_an)
fig.update_layout(showlegend=False,xaxis_title=None,yaxis_title=None)
dt.fig_save_or_show(fig, 'evenements_03_events_par_an')